In [293]:
import tensorflow as tf
import keras
from keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import TensorBoard
import os
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam

In [294]:
path = "gesture_data"

data = []
labels = []

for file_name in os.listdir(path):
    if file_name .endswith(".csv"):
        label = file_name.split("_")[0]
        file_path = os.path.join(path, file_name)

        df = pd.read_csv(file_path)
        data.append(df.values)
        labels.append(label)

In [295]:
np.array(data).shape, np.array(labels).shape

((36, 30, 63), (36,))

In [296]:
X = np.array(data)
y = np.array(labels)

In [297]:
label_encoder = LabelEncoder()
y_encoded = to_categorical(label_encoder.fit_transform(labels)) 

In [298]:
model = keras.Sequential([
    Input(shape=(30,63)),
    LSTM(64, return_sequences=True, activation='relu'),
    LSTM(128, return_sequences=True, activation='relu'),
    LSTM(64, return_sequences=False, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

In [299]:
adam = Adam(learning_rate=0.0001)
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [300]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(patience=10, restore_best_weights=True,)
history = model.fit(
    X,y_encoded,
    epochs=100,
    validation_split=0.2,
    batch_size=16,
    callbacks=callback
)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.2976 - loss: 1.0972 - val_accuracy: 0.3750 - val_loss: 1.0943
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3601 - loss: 1.0949 - val_accuracy: 0.3750 - val_loss: 1.0930
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3601 - loss: 1.0939 - val_accuracy: 0.3750 - val_loss: 1.0919
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3601 - loss: 1.0929 - val_accuracy: 0.3750 - val_loss: 1.0909
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3393 - loss: 1.0923 - val_accuracy: 0.3750 - val_loss: 1.0899
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3393 - loss: 1.0908 - val_accuracy: 0.3750 - val_loss: 1.0888
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3393 - loss: 1.0907 - val_accuracy: 0.3750 - val_loss: 1.0878
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3185 - loss: 1.0896 - val_accuracy: 0.3750 - val_loss: 1

In [301]:
model.save('gesture_classifier.h5')

In [302]:
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Class Mapping:")
for class_name, encoded_value in class_mapping.items():
    print(f"{class_name}: {encoded_value}")

Class Mapping:
faster: 0
resume: 1
stop: 2
